## A Critical Look at Cuckoo Search

This lab explores the common issue of seemingly new optimization algorithms that may not offer genuinely novel ideas. We'll first examine the original Cuckoo Search paper and then critically analyze it using the paper "An analysis of why cuckoo search does not bring any novel ideas to optimization." This exercise builds on our previous discussions of evolutionary algorithms like Differential Evolution and the pitfalls of poor research in fields like Neuroevolution, focusing here on the problem of redundant concepts in optimization.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from typing import Callable
from dataclasses import dataclass
from math import gamma

### Optimization Problems

This cell defines three common benchmark functions, Sphere, Rosenbrock, and Rastrigin, used to test optimization algorithms. We also used these functions earlier to evaluate Adam, Momentum, and CMA-ES.

In [7]:
def sphere(x: np.ndarray) -> float:
    return float(np.sum(x**2))


def rosenbrock(x: np.ndarray) -> float:
    return float(np.sum(100.0 * (x[1:] - x[:-1] ** 2.0) ** 2.0 + (1.0 - x[:-1]) ** 2.0))


def rastrigin(x: np.ndarray) -> float:
    A: float = 10.0
    return float(A * len(x) + np.sum(x**2 - A * np.cos(2 * np.pi * x)))

BOUNDS = [(-5, 5), (-5, 5)]

### Visualizing Search Dynamics

In [ ]:
def animate_cs(
    func: Callable[[np.ndarray], float],
    history: list[np.ndarray],
    bounds: list[tuple[float, float]] = BOUNDS,
    frames: int | None = None,
    filename: str = "cs_animation.gif",
) -> None:
    """
    Creates and saves a GIF showing how the CS population moves over generations.
    """
    if frames is None:
        frames = len(history)

    assert len(bounds) == 2, "This function only supports 2D visualization (expected 2 bounds)."
    x_bounds = (bounds[0][0], bounds[0][1])
    y_bounds = (bounds[1][0], bounds[1][1])

    x = np.linspace(x_bounds[0], x_bounds[1], 200)
    y = np.linspace(y_bounds[0], y_bounds[1], 200)
    X, Y = np.meshgrid(x, y)
    coords = np.vstack([X.ravel(), Y.ravel()]).T
    Z = np.array([func(pt) for pt in coords]).reshape(X.shape)

    fig, ax = plt.subplots(figsize=(8, 6))
    contour = ax.contourf(X, Y, Z, levels=20, cmap="viridis")
    fig.colorbar(contour, ax=ax)
    
    def init():
        scatter = ax.scatter([], [], s=20, color="red")
        return (scatter,)
    
    def update(i: int):
        ax.set_title(f"Generation {i}")
        pop = history[i]
        scatter = ax.scatter(pop[:, 0], pop[:, 1], s=20, color="red")
        return (scatter,)
    
    ax.set_xlim(x_bounds[0], x_bounds[1])
    ax.set_ylim(y_bounds[0], y_bounds[1])
    
    anim = animation.FuncAnimation(
        fig, update, init_func=init, frames=frames, interval=200, blit=True
    )
    
    writer = animation.PillowWriter(fps=5)
    anim.save(filename, writer=writer)
    plt.close(fig)
    print(f"Animation saved to {filename}")

### Exercise 1

Read [Cuckoo Search via Levy Flights](https://arxiv.org/pdf/1003.1594) with particular attention to Sections 2, 3, and 4. The primary focus for this exercise is Figure 1, which outlines the core pseudocode of the algorithm.

Your task is to implement the Cuckoo Search algorithm, using Figure 1 as your main reference. As the pseudocode is relatively high-level and lacks 
implementation details, you are encouraged to adopt a straightforward approach in your implementation.

**Action Item:** Document any ambiguities or unclear aspects you encounter in the algorithm description or pseudocode.

> Note: If you find the implementation too challenging or feel stuck, you may proceed directly to Exercise 2.

In [8]:
@dataclass
class CSResult:
    best_vector: np.ndarray
    best_value: float
    history: list[np.ndarray]  # History of populations for animation


class LevyFlight:
    def __init__(self, beta: float = 1.5):
        self.beta = beta

    def __call__(self, size: int) -> np.ndarray:
        sigma_u = (
            gamma(1 + self.beta)
            * np.sin(np.pi * self.beta / 2)
            / (gamma((1 + self.beta) / 2) * self.beta * 2 ** ((self.beta - 1) / 2))
        ) ** (1 / self.beta)
        sigma_v = 1
        u = np.random.normal(0, sigma_u, size)
        v = np.random.normal(0, sigma_v, size)
        step = u / np.abs(v) ** (1 / self.beta)
        return step


def cuckoo_search(
    func: Callable[[np.ndarray], float],
    bounds: list[tuple[float, float]] = BOUNDS,
    pop_size: int = 50,
    alpha: float = 1.0,
    beta: float = 1.5,
    p: float = 0.25,
    max_gen: int = 100,
) -> CSResult:
    """
    Implements the Cuckoo Search algorithm for global optimization.

    Parameters:
        func: Objective function to minimize. Takes a numpy array and returns a float.
        bounds: list of (min, max) pairs for each dimension.
        pop_size: Number of individuals in the population.
        alpha: Step-size scaling factor controlling the overall scale of the Lévy flights.
        beta: Exponent parameter for the Lévy distribution (typically in (1, 3]) that
            influences the heavy-tailed step-length distribution.
        p: Probability (in [0, 1]) that a host bird discovers an alien egg and abandons
            the corresponding nest—i.e., fraction of worse nests to be replaced each generation.
        max_gen: Maximum number of generations to evolve.
    """
    raise NotImplementedError("Not implemented")
    dimensions = len(bounds)
    lower_bounds = np.array([b[0] for b in bounds])
    upper_bounds = np.array([b[1] for b in bounds])

    # --- TODO: Exercise 1 - Initialization ---
    # Initialize the population: Create pop_size individuals.
    # Each individual is a vector of 'dimensions' length.
    # Use uniform distribution to sample the initial population.
    # --- End Exercise 1 ---

    for generation in range(max_gen):
        ...

### Test implemented CS

In [ ]:
result = cuckoo_search(sphere, bounds=BOUNDS, pop_size=50)

### Experiments

Run CS on all three problems: Sphere, Rosenbrock and Rastrigin. For each problem:
- Visualize the population dynamics over time to illustrate how the search space is explored and exploited.

### Exercise 2

Read [An analysis of why cuckoo search does not bring any novel ideas to optimization](https://www.sciencedirect.com/science/article/pii/S0305054822000442). Focus particularly on Sections 2 and 3. Section 2.3 provides a detailed description of the implemented Cuckoo Search algorithm. Carefully analyze and compare it with your own, highlighting any differences in assumptions, parameter settings, or algorithmic structure.

### Exercise 3

Read the Introduction of “An analysis of why cuckoo search does not bring any novel ideas to optimization.” Identify and outline three criteria proposed by the authors for evaluating the underlying metaphor of the algorithm. Critically reflect on these criteria, do you find them appropriate and sufficient? Can you suggest any additional criteria or alternative perspectives that might enrich the evaluation?

### Exercise 4

Read Section 4 of “An analysis of why cuckoo search does not bring any novel ideas to optimization.” Explain the main criticisms they raise against the Cuckoo Search algorithm. What fundamental issues do they identify, and how do these undermine the algorithm's novelty?

### Exercise 5

Analyze Figures 5 and 8 from [Large-scale Benchmarking of Metaphor-based Optimization Heuristics](https://arxiv.org/pdf/2402.09800). In these figures, Cuckoo Search is denoted as CS. Evaluate its performance relative to the CMA-ES variant (bipop) and Differential Evolution (DE). How does CS compare to these well-established algorithms in terms of optimization performance? Furthermore, critically consider whether performance alone is a sufficient criterion for evaluating optimization algorithms. What other factors should be taken into account?